In [4]:
import hyperopt
import tensorflow as tf
from hyperopt import fmin, tpe, hp
from tensorflow.keras.datasets import mnist
from keras import utils
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.models import Sequential
from keras.optimizers import SGD
from sklearn.datasets import make_classification
from scikeras.wrappers import KerasClassifier

### **Оптимизация гиперпараметров нейронной сети**

**Будем использовать набор данных MNIST, который состоит из полутоновых изображений рукописных цифр (0–9), размер которых составляет 28x28 пикселей. Каждый пиксель имеет 8 бит, поэтому его значение колеблется от 0 до 255**

### Преобразование данных

In [5]:
# Загрузка данных
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train.shape, y_train.shape, x_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28))

In [6]:
# Нормализуем значения пикселей (дать им значения от 0 до 1)

x_train, x_test = x_train / 255.0, x_test / 255.0

In [7]:
# Преобразуем массивы 2D-изображений в 1D (сглаживаем их). 
# Выполнить это можно используя изменение формы массива с помощью numpy.reshape()
# или keras.layers.Flatten который преобразует формат изображений из 2-мерного массива (размером 28 на 28 пикселей) 
# в 1-мерный массив размером 28 * 28 = 784 пикселей.

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [8]:
# One Hot Encodingy
y_train = utils.to_categorical(y_train, 10)
y_test =utils. to_categorical(y_test, 10)
y_test.shape, y_train.shape

((10000, 10), (60000, 10))

### Построение модели

In [9]:
def objective(params):
    input_dim = x_train.shape[1]
    model = Sequential([
        Dense(64, activation=tf.nn.relu, kernel_initializer='uniform', input_dim = input_dim),
        Dense(params['units'], kernel_initializer='uniform', activation=tf.nn.relu),
        Dense(10, kernel_initializer='uniform', activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model_history = model.fit(x_train, y_train,
                    batch_size=int(input_dim/100),
                    epochs=5,
                    verbose=1,
                    validation_split = 0.2)
    val_accuracy =  model_history.history['val_accuracy'][-1]
    
    return -val_accuracy 

**Определение пространства поиска**

In [10]:
space = {
    'units': hp.choice('units', [32, 64, 128]),  # Количество нейронов в слое
    'learning_rate': hp.loguniform('learning_rate', -5, -1)  # Логарифмический выбор скорости обучения
}

**Запуск поиска**

In [11]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)  # Количество экспериментов

Epoch 1/5                                             

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

job exception: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\Pavel_Pigal\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Pavel_Pigal\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Pavel_Pigal\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\Pavel_Pigal\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Python311\Lib\asyncio\base_events.py", line 604, in

  0%|          | 0/10 [00:02<?, ?trial/s, best loss=?]


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\Pavel_Pigal\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Pavel_Pigal\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Pavel_Pigal\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\Pavel_Pigal\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Python311\Lib\asyncio\base_events.py", line 

In [ ]:
print( model.evaluate(x_test, y_test) )

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

**Определение пространство поиска**

In [ ]:
space = {
    'units': hp.choice('units', [32, 64, 128]),  # Количество нейронов в слое
    'learning_rate': hp.loguniform('learning_rate', -5, -1)  # Логарифмический выбор скорости обучения
}

**Запуск поиска**

In [ ]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)  # Количество экспериментов

NameError: name 'objective' is not defined

### Grid Search

In [ ]:
import numpy as np
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification

In [ ]:
clf = KerasClassifier(
    model,
    loss="sparse_categorical_crossentropy",
    hidden_layer_dim=100,
)

clf.fit(x_train, y_train)
y_proba = clf.predict_proba(X_train)

TypeError: BaseWrapper._check_model_param.<locals>.final_build_fn() got an unexpected keyword argument 'hidden_layer_dim'

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', clf),
])

pipe.fit(X_train, y_train)
y_proba = pipe.predict_proba(X_train)

NameError: name 'clf' is not defined

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:


params = {
    "hidden_layer_dim": [50, 100, 200],
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
gs = RandomizedSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(X_train, y_train)
print(gs.best_score_, gs.best_params_)

NameError: name 'clf' is not defined

In [ ]:
print(f'Best Accuracy for {grid_result.best_score_:.4} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f'mean={mean:.4}, std={stdev:.4} using {param}')

NameError: name 'grid_result' is not defined